# Weather Data Accuracy Assessment
This notebook creates a pipeline which determines both the most accurate interpolation method for a series of weather station locations within Minnesota and the accuracy of the point data through comparison of known elevation data. The data is then stored in a file geodatabase and moved to an .sde database for creating real-time web maps in ArcGIS Online.

#### Import All Required Modules for this Lab

In [7]:
# Library for data manipulation and analysis
import pandas as pd

# Library for working with ArcGIS tools and functionalities
import arcpy
import arcgis


# Libraries for working with PostgreSQL
import psycopg2
from psycopg2 import sql

# Library for handling JSON data
import json

# Library for making HTTP requests and interacting with web services
import requests

# Library for interacting with the operating system, managing file paths, and executing system commands
import os

# Library for handling warnings generated during code execution
import warnings

# Library for displaying images in Jupyter Notebooks
from IPython.display import Image
from IPython.display import display

# Library for generating random numbers
import random

# Library for handling zip files and working with IO streams
import zipfile
import io

# Library for working with dates
from datetime import date

In [8]:
file_path = os.path.dirname(arcpy.mp.ArcGISProject('CURRENT').filePath)

# Change the current working directory to the extracted directory path
os.chdir(file_path)

# Set the workspace environment to the extracted directory path
arcpy.env.workspace = file_path

# Define a spatial reference with ID 26915 (likely UTM zone 15N)
spatial_ref = arcpy.SpatialReference(26915)

# Establish variables for project and map
project = arcpy.mp.ArcGISProject("CURRENT")
m = project.listMaps("Map")[0]

### Path to FGDB And SDE

In [9]:
# Establish SDE Connection via PGAdmin & Catalog Pane in ArcGIS Pro
sde = r"F:\1. UMN MGIS\1. Semesters\4th Semester\1. ArcGIS II\2. Labs\Lab 03\ArcII_Lab3\PostgreSQL-35-gis5572(postgres).sde"

#path to local database
local_gdb = r"F:\1. UMN MGIS\1. Semesters\4th Semester\1. ArcGIS II\2. Labs\Lab 03\ArcII_Lab3\ArcII_Lab3.gdb"

In [5]:
# Create database connection
dbname = 'PostgreSQL-35-gis5572(postgres).sde'
platform = 'POSTGRESQL'
user = 'postgres'
password = '*****'
instance = '35.238.64.215'
port = '5432'
auth = 'DATABASE_AUTH'
save = 'SAVE_USERNAME'
db = 'gis5572'

arcpy.management.CreateDatabaseConnection(
    out_folder_path = file_path,
    out_name = dbname,
    database_platform = platform,
    instance = instance,
    account_authentication = auth,
    username = user,
    password = password,
    save_user_pass = save,
    database = db
)

<Result 'F:\\1. UMN MGIS\\1. Semesters\\4th Semester\\1. ArcGIS II\\2. Labs\\Lab 03\\ArcII_Lab3\\PostgreSQL-35-gis5572(postgres).sde'>

### Minnesota State and Cities Boundary Boundary¶
by MN Geospatial Commons

State: https://gisdata.mn.gov/dataset/bdry-state-of-minnesota

Cities: https://gisdata.mn.gov/dataset/bdry-mn-city-township-unorg

In [5]:
# Paths to the input shapefiles and output geodatabase
mn_state_boundary = r"F:\1. UMN MGIS\1. Semesters\4th Semester\1. ArcGIS II\2. Labs\Lab 02\Data\MN County Boundaries\mn_county_boundaries"
mn_state_cities = r"F:\1. UMN MGIS\1. Semesters\4th Semester\1. ArcGIS II\2. Labs\Lab 02\Data\MN County Boundaries\shp_bdry_mn_city_township_unorg\city_township_unorg"

# Copy shapefiles to the geodatabase
arcpy.conversion.FeatureClassToGeodatabase(mn_state_boundary, local_gdb)
arcpy.conversion.FeatureClassToGeodatabase(mn_state_cities, local_gdb)

# Add copied shapefiles to the map
m.addDataFromPath(local_gdb + "\\" + os.path.basename(mn_state_boundary))
m.addDataFromPath(local_gdb + "\\" + os.path.basename(mn_state_cities))

print("Shapefiles copied to the file geodatabase.")

Shapefiles copied to the file geodatabase.


### WEATHER DATA QA/QC (Subject to change)

#### Setting Up Date Variables and Fetching Weather Data

In [20]:
# Establish variables for the dates
today = str(date.today())

day_num = int(today[8:10])
month_num = int(today[5:7])
year_num = int(today[0:4])
day = str(day_num)
month = str(month_num)
year = str(year_num)

# Acquire MN temperature data from Iowa State API in GeoJSON format
minnesota_temp = requests.get(r'https://mesonet.agron.iastate.edu/api/1/daily.geojson?network=MN_RWIS&date=' + today)
minn_json = minnesota_temp.json()

#### GeoJSON Weather Data Error Checking and Validation 

In [16]:
# Establish variables for the empty error list and the feature set in the GeoJSON
error_list = []
fs = minn_json['features']

# Create for loop iterating through each feature in the geojson
for f in fs:
    properties = f['properties']
    properties_list = [properties['precip'],properties['snow'],properties['snowd']]
    # Set variables equal to maximum and minimum temperature
    max_temp = properties['max_tmpf']
    min_temp = properties['min_tmpf']
    
    # Make sure the date of the incoming data is today
    if properties['date'] == today:
            
        # Create if statement for any empty temperature data
        if max_temp == None or min_temp == None:
            properties['error'] = 'Error: No temperature data available at this location'

        # Use monthly temperature records and add and subtract 5 from the record high and low respectively
        elif month_num == 1:
            if max_temp > 74:
                properties['error'] = 'Error: Maximum temperature is likely incorrect'
            if min_temp < -62:
                properties['error'] = 'Error: Minimum temperature is likely incorrect'
            else:
                properties['error'] = 'No Errors Found'
        elif month_num == 2:
            if max_temp > 81:
                properties['error'] = 'Error: Maximum temperature is likely incorrect'
            if min_temp < -65:
                properties['error'] = 'Error: Minimum temperature is likely incorrect'
            else:
                properties['error'] = 'No Errors Found'
        elif month_num == 3:
            if max_temp > 93:
                properties['error'] = 'Error: Maximum temperature is likely incorrect'
            if min_temp < -55:
                properties['error'] = 'Error: Minimum temperature is likely incorrect'
            else:
                properties['error'] = 'No Errors Found'
        elif month_num == 4:
            if max_temp > 106:
                properties['error'] = 'Error: Maximum temperature is likely incorrect'
            if min_temp < -27:
                properties['error'] = 'Error: Minimum temperature is likely incorrect'
            else:
                properties['error'] = 'No Errors Found'
        elif month_num == 5:
            if max_temp > 117:
                properties['error'] = 'Error: Maximum temperature is likely incorrect'
            if min_temp < -1:
                properties['error'] = 'Error: Minimum temperature is likely incorrect'
            else:
                properties['error'] = 'No Errors Found'
        elif month_num == 6:
            if max_temp > 115:
                properties['error'] = 'Error: Maximum temperature is likely incorrect'
            if min_temp < 10:
                properties['error'] = 'Error: Minimum temperature is likely incorrect'
            else:
                properties['error'] = 'No Errors Found'
        elif month_num == 7:
            if max_temp > 120:
                properties['error'] = 'Error: Maximum temperature is likely incorrect'
            if min_temp < 19:
                properties['error'] = 'Error: Minimum temperature is likely incorrect'
            else:
                properties['error'] = 'No Errors Found'
        elif month_num == 8:
            if max_temp > 115:
                properties['error'] = 'Error: Maximum temperature is likely incorrect'
            if min_temp < 16:
                properties['error'] = 'Error: Minimum temperature is likely incorrect'
            else:
                properties['error'] = 'No Errors Found'
        elif month_num == 9:
            if max_temp > 116:
                properties['error'] = 'Error: Maximum temperature is likely incorrect'
            if min_temp < 5:
                properties['error'] = 'Error: Minimum temperature is likely incorrect'
            else:
                properties['error'] = 'No Errors Found'
        elif month_num == 10:
            if max_temp > 103:
                properties['error'] = 'Error: Maximum temperature is likely incorrect'
            if min_temp < -11:
                properties['error'] = 'Error: Minimum temperature is likely incorrect'
            else:
                properties['error'] = 'No Errors Found'
        elif month_num == 11:
            if max_temp > 89:
                properties['error'] = 'Error: Maximum temperature is likely incorrect'
            if min_temp < -52:
                properties['error'] = 'Error: Minimum temperature is likely incorrect'
            else:
                properties['error'] = 'No Errors Found'
        elif month_num == 12:
            if max_temp > 79:
                properties['error'] = 'Error: Maximum temperature is likely incorrect'
            if min_temp < -62:
                properties['error'] = 'Error: Minimum temperature is likely incorrect'
            else:
                properties['error'] = 'No Errors Found'
        elif field in properties_list < 0:
            properties['error'] = 'Error: Negative values present in precipitation fields'
        else:
            properties['error'] = 'No Errors Found'
            
    # If the data's date is incorrect, print 'Incorrect date'
    else:
        properties['error'] = 'Error: Incorrect date'
        
# Print geojson
print(minn_json)

{'type': 'FeatureCollection', 'crs': {'type': 'name', 'properties': {'name': 'urn:ogc:def:crs:OGC:1.3:CRS84'}}, 'features': [{'type': 'Feature', 'properties': {'station': 'MN001', 'date': '2024-03-28', 'max_tmpf': 36.68, 'min_tmpf': 16.88, 'precip': None, 'max_gust': 10.6892, 'snow': None, 'snowd': None, 'min_rh': 40.0, 'max_rh': 87.0, 'max_dwpf': 17.276003, 'min_dwpf': 11.569978, 'min_feel': 11.282033, 'avg_feel': None, 'max_feel': 36.68, 'max_drct': None, 'precip_est': False, 'tmpf_est': False, 'max_gust_localts': '2024-03-28T12:20:00Z', 'temp_hour': None, 'avg_sknt': None, 'vector_avg_drct': None, 'min_rstage': None, 'max_rstage': None, 'id': 'MN001', 'name': 'Twin Lakes I-35 Mile Post 1', 'error': 'No Errors Found'}, 'geometry': {'type': 'Point', 'coordinates': [-93.354057312, 43.5083312988]}}, {'type': 'Feature', 'properties': {'station': 'MN002', 'date': '2024-03-28', 'max_tmpf': 29.48, 'min_tmpf': 10.58, 'precip': None, 'max_gust': 9.330454, 'snow': None, 'snowd': None, 'min_rh'

#### Data Processing and Conversion

In [26]:
# Create empty list for each necessary field
sta_id_list = []
date_list = []
max_temp_list = []
min_temp_list = []
precip_list = []
snow_list = []
snow_dep_list = []
sta_name_list = []
lon_list = []
lat_list = []
error_list = []

# Append correct data from the GeoJSON to each list
fs = minn_json['features'] 
for f in fs:
    sta_id_list.append(f['properties']['station'])
    date_list.append(f['properties']['date'])
    # Check for null values and replace them with 0
    max_temp_list.append(f['properties']['max_tmpf'] if f['properties']['max_tmpf'] is not None else 0)
    min_temp_list.append(f['properties']['min_tmpf'] if f['properties']['min_tmpf'] is not None else 0)
    precip_list.append(f['properties']['precip'] if f['properties']['precip'] is not None else 0)
    snow_list.append(f['properties']['snow'] if f['properties']['snow'] is not None else 0)
    snow_dep_list.append(f['properties']['snowd'] if f['properties']['snowd'] is not None else 0)
    sta_name_list.append(f['properties']['name'])
    lon_list.append(f['geometry']['coordinates'][0])
    lat_list.append(f['geometry']['coordinates'][1])
    try:
        error_list.append(f['properties']['error'])
    except KeyError:
        error_list.append('No Error Found')  # Append a default value when 'error' key is missing

# Create a dictionary for pandas to use to create a dataframe
gdf_dict = {
    "station_id":sta_id_list,
    "date":date_list,
    "max_daily_temp_F":max_temp_list,
    "min_daily_temp_F":min_temp_list,
    "precip_in":precip_list,
    "snow_in":snow_list,
    "snow_depth_in":snow_dep_list,
    "station_name":sta_name_list,
    "longitude":lon_list,
    "latitude":lat_list,
    "error":error_list
}

# Set a variable equal to the date without the dash marks for saving
today_no_dash = today.replace('-','')

# Establish a variable for where the temperature logs will be saved
temp_log_path = r'F:\1. UMN MGIS\1. Semesters\4th Semester\1. ArcGIS II\2. Labs\Lab 03\Data\MN_temp_log_' + today_no_dash + '.csv'

# Create a .csv file using the pandas dataframe and add the .csv to the map
station_df = pd.DataFrame(gdf_dict)
station_df.to_csv(temp_log_path)
m.addDataFromPath(temp_log_path)

# Establish a variable for the .csv file and use the latitude and longitude values to create a point feature class
csv_file = 'MN_temp_log_' + today_no_dash + '.csv'
weather_fc = 'MN_temp_log_' + today_no_dash

arcpy.management.ConvertCoordinateNotation(
    in_table = csv_file,
    out_featureclass = os.path.join(local_gdb, weather_fc),
    x_field = 'longitude',
    y_field = 'latitude',
    input_coordinate_format = 'DD_2',
    output_coordinate_format = 'UTM_ZONES',
    spatial_reference = spatial_ref
)

<Result 'F:\\1. UMN MGIS\\1. Semesters\\4th Semester\\1. ArcGIS II\\2. Labs\\Lab 03\\ArcII_Lab3\\ArcII_Lab3.gdb\\MN_temp_log_20240328'>

#### Export from local FGDB to PostgreSQL

In [27]:
# Function to upload feature class to SDE
def upload_feature_class_to_sde(input_feature_class, output_sde_connection, output_sde_feature_class):
    try:
        arcpy.FeatureClassToFeatureClass_conversion(input_feature_class, output_sde_connection, output_sde_feature_class)
        print(f"Feature class '{output_sde_feature_class}' uploaded successfully to SDE.")
    except Exception as e:
        print(f"Error uploading feature class to SDE: {str(e)}")

# Paths and parameters
input_feature_class = r'F:\1. UMN MGIS\1. Semesters\4th Semester\1. ArcGIS II\2. Labs\Lab 03\ArcII_Lab3\ArcII_Lab3.gdb\MN_temp_log_20240328'
output_sde_connection = r'F:\1. UMN MGIS\1. Semesters\4th Semester\1. ArcGIS II\2. Labs\Lab 03\ArcII_Lab3\PostgreSQL-35-gis5572(postgres).sde'
output_sde_feature_class = 'mnweather'

# Upload feature class to SDE
upload_feature_class_to_sde(input_feature_class, output_sde_connection, output_sde_feature_class)

Feature class 'mnweather' uploaded successfully to SDE.


In [ ]:
sde = {
    "server": "35.226.160.201",
    "database": "gis5572",
    "user": "postgres",
    "password": "Hyderabad43%",
    "port": "5432"
}

### Pulling Data from SDE to FGDB Data 

In [48]:
# Define the SDE feature class path
sde_feature_class = "gis5572.postgres.mn_clean_weather"

# Define the local feature class path
local_feature_class = os.path.join(local_gdb, "mn_clean_weather")

# Process: Copy Features
arcpy.CopyFeatures_management(r"F:\1. UMN MGIS\1. Semesters\4th Semester\1. ArcGIS II\2. Labs\Lab 03\ArcII_Lab3\PostgreSQL-35-gis5572(postgres).sde\gis5572.postgres.mn_clean_weather", local_feature_class)

<Result 'F:\\1. UMN MGIS\\1. Semesters\\4th Semester\\1. ArcGIS II\\2. Labs\\Lab 03\\ArcII_Lab3\\ArcII_Lab3.gdb\\mn_clean_weather'>

### Create Random Sample from QAQC'd Data

Sampling 50% of the data randomly is beneficial because it reduces computational burden, maintains key dataset characteristics, and saves costs od saving and processing data. It ensures representative insights while mitigating bias, making analysis more efficient and reliable. 

In [18]:
mn_clean_weather = r"F:\1. UMN MGIS\1. Semesters\4th Semester\1. ArcGIS II\2. Labs\Lab 03\ArcII_Lab3\ArcII_Lab3.gdb\mn_weather"

def create_sample(input_feature_class, x_field, y_field, z_field, percent, gdb_path, out_feature_class):
    
    print('Creating feature class "' + out_feature_class + '"...')

    all_rows = []
    with arcpy.da.SearchCursor(in_table = input_feature_class, field_names = [z_field, x_field, y_field]) as cursor:
        for row in cursor:
            Z = row[0]
            X = row[1]
            Y = row[2]
            all_rows.append([Z, X, Y])

    total_rows = len(all_rows)
    sample_num = int(total_rows * percent)
    removed_num = total_rows - sample_num

    random.shuffle(all_rows)
    
    Z_list = []
    X_list = []
    Y_list = []

    for row in all_rows:
        Z_list.append(row[0])
        X_list.append(row[1])
        Y_list.append(row[2])

    df_list = [Z_list, X_list, Y_list]
    for l in df_list:
        del l[-removed_num:]

    rand_dict = {
        'Z':Z_list,
        'X':X_list,
        'Y':Y_list
    }

    random_sample_df = pd.DataFrame(rand_dict)
    
    # Convert table to sedf
    sedf = arcgis.GeoAccessor.from_xy(
        df = random_sample_df, 
        x_column = "X",
        y_column = "Y"
    )

    # Convert sedf to feature class
    sedf.spatial.to_featureclass(location=os.path.join(gdb_path, out_feature_class))
    
    map_epsg = spatial_ref.factoryCode
    fc_epsg = arcpy.Describe(out_feature_class).spatialReference.factoryCode
    
    if map_epsg != fc_epsg:
        print("Projecting...")
        
        arcpy.management.DefineProjection(
            in_dataset = out_feature_class,
            coor_system = spatial_ref
        )
        
    arcpy.management.AddField(
        in_table = out_feature_class,
        field_name = z_field,
        field_type = 'FLOAT'
    )
    
    arcpy.management.CalculateField(
        in_table = out_feature_class,
        field = z_field,
        expression = '!z!'
    )
    
    arcpy.management.DeleteField(
        in_table = out_feature_class,
        drop_field = 'z'
    )
        
    print('Done')

In [19]:
# Define the well-known ID (WKID) of the spatial reference (e.g., EPSG code)
# spatial_ref_wkid = 4326  

In [20]:
# Create a feature class with 50% of random points sampled
create_sample(
    input_feature_class=mn_clean_weather,
    x_field='x',
    y_field='y',
    z_field='max_tmpf',
    percent=0.5,
    gdb_path=local_gdb,
    out_feature_class='Random_Sample_50_temp',
)

Creating feature class "Random_Sample_50_temp"...
Projecting...
Done


### Quality Assesment of Each Model

In [29]:
# Specify the full path to the dataset
dataset_path = r'F:\1. UMN MGIS\1. Semesters\4th Semester\1. ArcGIS II\2. Labs\Lab 03\ArcII_Lab3\ArcII_Lab3.gdb\Random_Sample_50_temp'

# Perform exploratory interpolation
arcpy.ga.ExploratoryInterpolation(
    in_features=dataset_path,
    value_field='max_tmpf',
    out_cv_table=os.path.join(local_gdb, 'stats_table_weather'),
    out_geostat_layer=None,
    interp_methods=['ORDINARY_KRIGING', 'UNIVERSAL_KRIGING', 'EBK'], #Empirical Bayesian Kriging
    comparison_method='SINGLE',
    criterion='ACCURACY',
    criteria_hierarchy='ACCURACY PERCENT #',
    weighted_criteria='ACCURACY 1',
    exclusion_criteria=None
)

<Result 'F:\\1. UMN MGIS\\1. Semesters\\4th Semester\\1. ArcGIS II\\2. Labs\\Lab 03\\ArcII_Lab3\\ArcII_Lab3.gdb\\stats_table_weather'>

### Interpolating the Weather Data using 3 Techniques

In [39]:
# KRINGING - ORDINARY
with arcpy.EnvManager(scratchWorkspace=r"F:\1. UMN MGIS\1. Semesters\4th Semester\1. ArcGIS II\2. Labs\Lab 03\ArcII_Lab3\ArcII_Lab3.gdb"):
    mnweather_KrigOrd = arcpy.sa.Kriging(
        in_point_features=r"Weather Data\RanSample_50_tem",
        z_field="max_tmpf",
        kriging_model="Spherical # # # #",
        cell_size=3000, # that's 3000 meters or 3 kms as I am using NAD1983 Zone 15N
        search_radius="VARIABLE 12",
        out_variance_prediction_raster=None
    )
    mnweather_KrigOrd.save(r"F:\1. UMN MGIS\1. Semesters\4th Semester\1. ArcGIS II\2. Labs\Lab 03\ArcII_Lab3\ArcII_Lab3.gdb\mnweather_KrigOrd")

In [42]:
# KRINGING - UNIVERSAL

with arcpy.EnvManager(scratchWorkspace=r"F:\1. UMN MGIS\1. Semesters\4th Semester\1. ArcGIS II\2. Labs\Lab 03\ArcII_Lab3\ArcII_Lab3.gdb"):
    mnweather_KrigUni = arcpy.sa.Kriging(
        in_point_features=r"Weather Data\RanSample_50_tem",
        z_field="max_tmpf",
        kriging_model="LinearDrift 2219.396320 # # #",
        cell_size=3000, # that's 3000 meters or 3 kms
        search_radius="VARIABLE 12",
        out_variance_prediction_raster=None
    )
    mnweather_KrigUni.save(r"F:\1. UMN MGIS\1. Semesters\4th Semester\1. ArcGIS II\2. Labs\Lab 03\ArcII_Lab3\ArcII_Lab3.gdb\mnweather_KrigUni")

In [41]:
# EMPIRICAL BAYESIAN KRIGING

arcpy.ga.EmpiricalBayesianKriging(
    in_features=r"Weather Data\RanSample_50_tem",
    z_field="max_tmpf",
    out_ga_layer=None,
    out_raster=r"F:\1. UMN MGIS\1. Semesters\4th Semester\1. ArcGIS II\2. Labs\Lab 03\ArcII_Lab3\ArcII_Lab3.gdb\mnweather_EBK",
    cell_size=3000, # that's 3000 meters or 3 kms
    transformation_type="NONE",
    max_local_points=100,
    overlap_factor=1,
    number_semivariograms=100,
    search_neighborhood="NBRTYPE=StandardCircular RADIUS=207149.441825675 ANGLE=0 NBR_MAX=15 NBR_MIN=10 SECTOR_TYPE=ONE_SECTOR",
    output_type="PREDICTION",
    quantile_value=0.5,
    threshold_type="EXCEED",
    probability_threshold=None,
    semivariogram_model_type="POWER"
)

<Result ''>

### Converting Interpolated Raster into Points And Exporting to SDE

In [15]:
# Step 1: Downsample Raster
def downsample_raster(input_raster, output_raster, cell_size):
    arcpy.Resample_management(input_raster, output_raster, cell_size)

# Step 2: Convert Raster to Points
def raster_to_points(input_raster, output_points):
    arcpy.RasterToPoint_conversion(input_raster, output_points, "VALUE")

# Step 3: Upload Points to SDE
def upload_points_to_sde(input_points, output_sde_connection, output_sde_feature_class):
    arcpy.FeatureClassToFeatureClass_conversion(input_points, output_sde_connection, output_sde_feature_class)

In [16]:
# Paths and parameters for each raster
raster_paths = [
    r'F:\1. UMN MGIS\1. Semesters\4th Semester\1. ArcGIS II\2. Labs\Lab 03\ArcII_Lab3\ArcII_Lab3.gdb\mnweather_EBK',
    r'F:\1. UMN MGIS\1. Semesters\4th Semester\1. ArcGIS II\2. Labs\Lab 03\ArcII_Lab3\ArcII_Lab3.gdb\mnweather_KrigOrd',
    r'F:\1. UMN MGIS\1. Semesters\4th Semester\1. ArcGIS II\2. Labs\Lab 03\ArcII_Lab3\ArcII_Lab3.gdb\mnweather_KrigUni'
]
output_downsampled_rasters = [
    r'F:\1. UMN MGIS\1. Semesters\4th Semester\1. ArcGIS II\2. Labs\Lab 03\ArcII_Lab3\downsampled_EBK.tif',
    r'F:\1. UMN MGIS\1. Semesters\4th Semester\1. ArcGIS II\2. Labs\Lab 03\ArcII_Lab3\downsampled_KrigOrd.tif',
    r'F:\1. UMN MGIS\1. Semesters\4th Semester\1. ArcGIS II\2. Labs\Lab 03\ArcII_Lab3\downsampled_KrigUni.tif'
]
output_points = [
    r'F:\1. UMN MGIS\1. Semesters\4th Semester\1. ArcGIS II\2. Labs\Lab 03\ArcII_Lab3\EBK_temp_points.shp',
    r'F:\1. UMN MGIS\1. Semesters\4th Semester\1. ArcGIS II\2. Labs\Lab 03\ArcII_Lab3\KrigOrd_temp_points.shp',
    r'F:\1. UMN MGIS\1. Semesters\4th Semester\1. ArcGIS II\2. Labs\Lab 03\ArcII_Lab3\KrigUni_temp_points.shp'
]
output_sde_connection = r'F:\1. UMN MGIS\1. Semesters\4th Semester\1. ArcGIS II\2. Labs\ArcPro\Lab2\ArcII_Lab2\PostgreSQL-35-gis5572(postgres).sde'

output_sde_feature_classes = [
    'EBK_temp_points',
    'KrigOrd_temp_points',
    'KrigUni_temp_points'
]

In [17]:
# Loop through each raster
for i in range(len(raster_paths)):
    # Step 1: Downsample Raster
    downsample_raster(raster_paths[i], output_downsampled_rasters[i], "15000")

    # Step 2: Convert Raster to Points
    raster_to_points(output_downsampled_rasters[i], output_points[i])

    # Step 3: Upload Points to SDE
    upload_points_to_sde(output_points[i], output_sde_connection, output_sde_feature_classes[i])

print("The entire 3 step process is completed.")

The entire 3 step process is completed.


In [18]:
import arcpy

# List of shapefiles to be exported
shapefiles = [
    r'F:\1. UMN MGIS\1. Semesters\4th Semester\1. ArcGIS II\2. Labs\Lab 03\ArcII_Lab3\EBK_temp_points.shp',
    r'F:\1. UMN MGIS\1. Semesters\4th Semester\1. ArcGIS II\2. Labs\Lab 03\ArcII_Lab3\KrigOrd_temp_points.shp',
    r'F:\1. UMN MGIS\1. Semesters\4th Semester\1. ArcGIS II\2. Labs\Lab 03\ArcII_Lab3\KrigUni_temp_points.shp'
]

# Output geodatabase
output_gdb = r'F:\1. UMN MGIS\1. Semesters\4th Semester\1. ArcGIS II\2. Labs\Lab 03\ArcII_Lab3\ArcII_Lab3.gdb'

# Export shapefiles to the file geodatabase
for shapefile in shapefiles:
    arcpy.conversion.FeatureClassToGeodatabase(shapefile, output_gdb)

print("Shapefiles exported to the file geodatabase.")

Shapefiles exported to the file geodatabase.


### Differences Between “Ground Truth” And Model Prediction

In [19]:
def differences_temp(true_points, interpolated_points):
    try:
        # Get the name of interpolation method as part of the output point layer
        interpolation = os.path.basename(interpolated_points)
        
        # Spatial join
        output_join = f"diff_{interpolation}"
        arcpy.analysis.SpatialJoin(true_points, interpolated_points, output_join, "JOIN_ONE_TO_ONE", "KEEP_ALL", "", "CLOSEST")
        
        # Calculate difference
        fields = ["max_tmpf", "grid_code", "Diff"]
        arcpy.management.AddField(output_join, "Diff", "DOUBLE")
        
        with arcpy.da.UpdateCursor(output_join, fields) as cursor:
            for row in cursor:
                row[2] = row[0] - row[1]  # Calculate difference: "max_tmpf" - "grid_code"
                cursor.updateRow(row)
        
        # Delete unnecessary fields
        arcpy.management.DeleteField(output_join, "Join_Count;TARGET_FID;pointid", "DELETE_FIELDS")
        print(f"Difference {interpolation} points layer created.")
    except Exception as e:
        print(f"An error occurred: {str(e)}")

In [20]:
# Set paths to true points and interpolated points
true_points = r'F:\1. UMN MGIS\1. Semesters\4th Semester\1. ArcGIS II\2. Labs\Lab 03\ArcII_Lab3\ArcII_Lab3.gdb\RanSample_50_tem'
interpolated_points = [
    r'F:\1. UMN MGIS\1. Semesters\4th Semester\1. ArcGIS II\2. Labs\Lab 03\ArcII_Lab3\ArcII_Lab3.gdb\EBK_temp_points',
    r'F:\1. UMN MGIS\1. Semesters\4th Semester\1. ArcGIS II\2. Labs\Lab 03\ArcII_Lab3\ArcII_Lab3.gdb\KrigOrd_temp_points',
    r'F:\1. UMN MGIS\1. Semesters\4th Semester\1. ArcGIS II\2. Labs\Lab 03\ArcII_Lab3\ArcII_Lab3.gdb\KrigUni_temp_points'
]

In [21]:
# Run differences_temp function for each interpolated point layer
for points in interpolated_points:
    differences_temp(true_points, points)

Difference EBK_temp_points points layer created.
Difference KrigOrd_temp_points points layer created.
Difference KrigUni_temp_points points layer created.


In [25]:
import arcpy

# List of shapefiles to be exported
shapefiles = [
r'F:\1. UMN MGIS\1. Semesters\4th Semester\1. ArcGIS II\2. Labs\Lab 03\ArcII_Lab3\diff_KrigUni_temp_points.shp',
    r'F:\1. UMN MGIS\1. Semesters\4th Semester\1. ArcGIS II\2. Labs\Lab 03\ArcII_Lab3\diff_KrigOrd_temp_points.shp',
    r'F:\1. UMN MGIS\1. Semesters\4th Semester\1. ArcGIS II\2. Labs\Lab 03\ArcII_Lab3\diff_EBK_temp_points.shp'
]

# Output geodatabase
output_gdb = r'F:\1. UMN MGIS\1. Semesters\4th Semester\1. ArcGIS II\2. Labs\Lab 03\ArcII_Lab3\ArcII_Lab3.gdb'

# Export shapefiles to the file geodatabase
for shapefile in shapefiles:
    arcpy.conversion.FeatureClassToGeodatabase(shapefile, output_gdb)

print("Shapefiles exported to the file geodatabase.")

Shapefiles exported to the file geodatabase.


### Exporting to SDE

In [27]:
points_list = ['diff_KrigOrd_temp_points']

for points in points_list:

    arcpy.conversion.FeatureClassToGeodatabase(
        Input_Features = points,
        Output_Geodatabase = sde
    )